In [1]:
# USEFUL FUNCTIONS

# results = sp.search(q='urbancoyotes', limit=20)
# for idx, track in enumerate(results['tracks']['items']):
#    print(idx, track['name'])
    
# import json
# playlist_id = 'spotify:user:spotifycharts:playlist:37i9dQZEVXbJiZcmkrIHGU'
# results = sp.playlist(playlist_id)
# print(json.dumps(results, indent=4))

# birdy = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
# results = spotify.artist_albums(birdy, album_type='album')
# albums = results['items']
# while results['next']:
#     results = spotify.next(results)
#     albums.extend(results['items'])

# for album in albums:
#     print(album['name'])

# artist = sp.artist(birdy)
# print(artist)

# print("")
# user = sp.user('kolsen118')
# print(user)

In [3]:
# We used Spotipy's documentation
# https://spotipy.readthedocs.io/en/2.12.0/

In [4]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from collections import Counter

%env SPOTIPY_CLIENT_ID = 3543bc68b7a64fa4896cb0c1d2499e33
%env SPOTIPY_CLIENT_SECRET = b711f5fe2e9a417c89247f7036c9c069

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

env: SPOTIPY_CLIENT_ID=3543bc68b7a64fa4896cb0c1d2499e33
env: SPOTIPY_CLIENT_SECRET=b711f5fe2e9a417c89247f7036c9c069


In [11]:
#prints number of playlists with corona_terms in title/description
#this is helpful bc it helps us see ahead of time how huge the dataset is

terms = ["Quarantine"]
for term in terms:
    playlists = sp.search('q="{}"'.format(term), type='playlist')
    number_playlists = playlists['playlists']['total']
    print(number_playlists)

288873


In [7]:
#function that creates a list with the playlist name, URI & total number of tracks
def extract_playlist(x,list_):
    for i in x['playlists']['items']:
        list_.append({"name" : i['name'], "total":i["tracks"]['total'], "uri":i["uri"]})

In [12]:
#create a list of playlists
list_of_playlists = []

for term in terms:
    
    #use index (i) which are multiples of 50 -- random sample bc of search limitation
    #spotify can only give you 2000 results at a time
    #random sample of 40 playlists
    for i in range(0,2000,50):
        
        try:
            #create variable init_data that has random sample of playlists with terms
            playlist_data = sp.search('q="{}"'.format(term), type='playlist', limit=50, offset=i)
                        #limit = 50, can only return up to 50 items, random sample of 50 songs
                        #type = playlist, only return playlists
                        #offset = i, the index of the first item to return
            
            extract_playlist(playlist_data,list_of_playlists)
            #above function appends init_data to list_of_playlists
        
        except:
            print("Error")

In [15]:
#dataset is length 2000 --> sample of 40 playlists, from each playlist, sample of 50 songs
len(list_of_playlists)

2000

In [16]:
#creates dataframe with list of playlists, dropping duplicates & delete their index
#only need to drop duplicates if you use multiple terms, i.e. Coronavirus AND Quarantine
playlists = pd.DataFrame(list_of_playlists).drop_duplicates().reset_index(drop=True)

In [17]:
#creates csv file from df
playlists.to_csv("playlists.csv")

In [21]:
#for a random sample of playlists, takes random sample of tracks, creates a list of tracks and their uris
tracks = []

#using each uri code listed in the playlist.csv
for uri in playlists['uri']:
        
        length = sp.playlist_tracks(uri)['total']
        #playlist_tracks(parameter) is playlist id, aka uri
        #length is the playlist length of each individual playlist
    
        for i in range(0,length, 50):
            #from 0 to playlist length, take every other 50th index
            
            playlist_data = sp.playlist_tracks(uri, limit=50, offset=i)
            #playlist_tracks get URI of tracks in playlists, input URI
            #playlist_data now has playlist track data
            #replacing old data
            
            try:
                for k in playlist_data['items']:
                    tracks.append(k['track']["uri"])
                    
            except:
                pass

In [22]:
len(tracks)

266309

In [23]:
#dataframe with individual track uri and count
df = pd.DataFrame.from_dict(Counter(tracks), orient='index').reset_index().rename(columns={"index":"uri", 0:"count"})

In [24]:
df

,uri,count
0,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,120
1,spotify:track:0VgkVdmE4gld66l8iyGjgx,56
2,spotify:track:2NxIIb2OZ1DSbfWAH0W47B,3
3,spotify:track:2xLMifQCjDGFmkHkpNLD9h,111
4,spotify:track:66s45uMhk7Y4z0xUgESdm3,26
...,...,...
130801,spotify:track:2s9Q0MX0Fhqh1GLpTtaucm,1
130802,spotify:track:3LESrNfSJ35jNjl80aKlY7,1
130803,spotify:track:6mUR50J1s6g4GaP8krxOzk,1
130804,spotify:track:4rm1WzJCTpwR3QC0xNjkNP,1


In [34]:
#useful stats to help us understand frequency of the popular tracks
#to know when we should cut off our data/if necessary
df[(df['count']>4) & (df['count']<2000)].describe()

,count
count,8944.000000
mean,11.926431
std,11.723794
min,5.000000
25%,6.000000
50%,8.000000
75%,13.000000
max,176.000000


In [52]:
#creates a csv file of the unfiltered track data
df.to_csv("tracks_list_unfiltered.csv")

In [54]:
#cuts off any tracks that appear less than twice (CAN CHANGE)
#this is useful bc we don't want tracks that don't appear frequently
sliced_df = df[df['count']>4]
len(sliced_df)

8944

In [55]:
#filters important variables, makes the dataframe usable
def filter_data(x):
    data = sp.track(x['uri'])
    #sets data variable for each track
    
    name = data['name']
    pop = data['popularity']
    date = data['album']['release_date']
    explic = data['explicit']
    
    features = sp.audio_features(x['uri'])
    #uses spotify function to access audio features
    
    return {**x, 
            "name" : name, 
            'release_date': date, 
            "popularity" : pop,
            'explicit' : explic, 
            **features[0]}

In [56]:
final_df = sliced_df.apply(lambda x: filter_data(x), axis=1, result_type='expand')

In [57]:
final_df

,uri,count,name,release_date,popularity,explicit,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,120,Toxic,2003-11-13,80,False,0.774,0.838,5,-3.914,...,0.02500,0.2420,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4
1,spotify:track:0VgkVdmE4gld66l8iyGjgx,56,Mask Off,2017-06-30,82,True,0.833,0.434,2,-8.795,...,0.02190,0.1650,0.281,150.062,audio_features,0VgkVdmE4gld66l8iyGjgx,https://api.spotify.com/v1/tracks/0VgkVdmE4gld...,https://api.spotify.com/v1/audio-analysis/0Vgk...,204600,4
3,spotify:track:2xLMifQCjDGFmkHkpNLD9h,111,SICKO MODE,2018-08-03,92,True,0.834,0.730,8,-3.714,...,0.00000,0.1240,0.446,155.008,audio_features,2xLMifQCjDGFmkHkpNLD9h,https://api.spotify.com/v1/tracks/2xLMifQCjDGF...,https://api.spotify.com/v1/audio-analysis/2xLM...,312820,4
4,spotify:track:66s45uMhk7Y4z0xUgESdm3,26,Unwell - 2007 Remaster,2007-09-29,63,False,0.442,0.793,9,-5.152,...,0.00000,0.3280,0.437,80.985,audio_features,66s45uMhk7Y4z0xUgESdm3,https://api.spotify.com/v1/tracks/66s45uMhk7Y4...,https://api.spotify.com/v1/audio-analysis/66s4...,237267,4
5,spotify:track:5veJDT0MLsLbhYsx42GXUD,91,Don't Stand So Close To Me,1980-10-03,71,False,0.799,0.506,2,-9.414,...,0.00107,0.0365,0.518,140.309,audio_features,5veJDT0MLsLbhYsx42GXUD,https://api.spotify.com/v1/tracks/5veJDT0MLsLb...,https://api.spotify.com/v1/audio-analysis/5veJ...,242667,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126861,spotify:track:7FNIEfhhBlzsZFxCwI5Nwh,29,A Little Less Conversation - JXL Radio Edit Remix,2002-09-24,60,False,0.590,0.976,11,-4.866,...,0.29300,0.1550,0.686,114.996,audio_features,7FNIEfhhBlzsZFxCwI5Nwh,https://api.spotify.com/v1/tracks/7FNIEfhhBlzs...,https://api.spotify.com/v1/audio-analysis/7FNI...,213240,4
126862,spotify:track:0TB7xPRIQ6sZqH8q50maWh,29,Wreckin' Bar (Ra Ra Ra),2011-03-11,51,False,0.404,0.937,8,-3.440,...,0.00254,0.2710,0.776,89.858,audio_features,0TB7xPRIQ6sZqH8q50maWh,https://api.spotify.com/v1/tracks/0TB7xPRIQ6sZ...,https://api.spotify.com/v1/audio-analysis/0TB7...,81720,4
126863,spotify:track:43n7zMxy0C4OeFIWLWOjcN,29,Fallowfield Hillbilly,2008-01-01,41,False,0.328,0.938,1,-2.634,...,0.00000,0.0327,0.624,162.756,audio_features,43n7zMxy0C4OeFIWLWOjcN,https://api.spotify.com/v1/tracks/43n7zMxy0C4O...,https://api.spotify.com/v1/audio-analysis/43n7...,181560,4
126864,spotify:track:4DRsSlnqJJGvskBWbT1Voy,29,Satellite - Single Edit,2019-06-21,46,False,0.740,0.697,0,-6.545,...,0.03780,0.2010,0.963,127.984,audio_features,4DRsSlnqJJGvskBWbT1Voy,https://api.spotify.com/v1/tracks/4DRsSlnqJJGv...,https://api.spotify.com/v1/audio-analysis/4DRs...,201336,4


In [58]:
#makes a csv with the final dataset
final_df.to_csv("tracks_list.csv",)